### AIM : to recommend the movie to the user based on his/her input based on

##### 1) content based recommendation System
##### 2) certain amount of popularity based recommendation System


### WORK FLOW:  

##### 1) Data
##### 2) Data Preprocesssing
##### 3) Feature Extraction               (convert the textaul data into feature vectors)
##### 4) Input from the user             ( based on this we can suggest then which movies they can watch)
##### 5) Cosine Similarity Algortihm (used to find similarity btw vectors)
##### 6) Get the list of Similar movies and recommend tothe user

In [1]:
# import some libraries

import pandas as pd    # to import dataset
import difflib      # to get the closest match to the given input
from sklearn.feature_extraction.text import TfidfVectorizer  # to convert textual data into numerical data means feature vectors
from sklearn.metrics.pairwise import cosine_similarity   # to calculate the similarity scores btw input feature and dataset features and make prediction based on that


### Data collection and Preprocessing

In [2]:
# lodaing the dataset from csv to pandas dataframe

df = pd.read_csv("movies.csv")
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
df.shape

(4803, 24)

So there are total 4803 movies in the list and 24 features

In [4]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [5]:
# selecting the relevant features for recommendation  this is basically for content based recommendation

selected_features = ['genres','keywords','tagline','cast','director']
selected_features

['genres', 'keywords', 'tagline', 'cast', 'director']

In [6]:
# to check how many null values are there in the dataset

df.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

So in our "selected features" 
Genres : 28 null values
Keywords : 412 null values
Tagline : 844 null values
Cast : 43
Director : 30

Techniques to fill this null values: we can replace all the null values with the empty string
we can't use mean imputation or mode becasue each movie will be unique and this will lead to false data

In [7]:
# replacing all the null values with the empty string

df = df.fillna('')

df.isnull().sum()

index                   0
budget                  0
genres                  0
homepage                0
id                      0
keywords                0
original_language       0
original_title          0
overview                0
popularity              0
production_companies    0
production_countries    0
release_date            0
revenue                 0
runtime                 0
spoken_languages        0
status                  0
tagline                 0
title                   0
vote_average            0
vote_count              0
cast                    0
crew                    0
director                0
dtype: int64

As you can see there is no null values now

In [8]:
selected_features

['genres', 'keywords', 'tagline', 'cast', 'director']

In [9]:
# now let's combine all the values of selected features

combined_features = df['genres']+' '+df['keywords']+' '+df['tagline']+' '+df['cast']+' '+df['director'] 

combined_features


0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [10]:
# now let's convert that combined_fatures data which is string into feature vectors means numerical values

vectorizer = TfidfVectorizer()   # creating an instance of TfidfVectorixer()

feature_vectors = vectorizer.fit_transform(combined_features)

print(feature_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

### Cosine Similarity

In [11]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


### Getting Input from user

In [13]:
movie_name = input(" Enter the name of your favourite movie : ")


 Enter the name of your favourite movie : The Dark Knight Rises	


In [14]:
# storing all the movie names present in the datest into a list

movies_list = df['title'].tolist()     # tolist() function is used to convet the values into list

movies_list

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [15]:
# finding the close matches movies for the movie name given by the user

close_match = difflib.get_close_matches(movie_name, movies_list)  # this difflib library using .get_close_matches function to get the name which is matching to user's input name

close_match

['The Dark Knight Rises',
 'The Dark Knight',
 'Batman: The Dark Knight Returns, Part 2']

In [16]:
# now the closest match will be the first movie in the list 
closest_match = close_match[0]
closest_match

'The Dark Knight Rises'

In [17]:
# find the index of closest match movie
index_of_the_movie = df[df.title == closest_match]['index'].values[0]
index_of_the_movie

3

In [18]:
# getting a list of similar movies

# enumerate function will act as a loop which will itereate into each item on the list
similarity_score = list(enumerate(similarity[index_of_the_movie])) 



similarity_score

[(0, 0.012520204623868908),
 (1, 0.020741502866825225),
 (2, 0.05179498007149405),
 (3, 1.0000000000000002),
 (4, 0.006717078317903558),
 (5, 0.045966370176512346),
 (6, 0.0),
 (7, 0.00696281041487878),
 (8, 0.03210611152276799),
 (9, 0.09959599730675214),
 (10, 0.11991876777277595),
 (11, 0.07201005831409965),
 (12, 0.007043386364464863),
 (13, 0.01154196679223832),
 (14, 0.11783709419157855),
 (15, 0.0),
 (16, 0.006602224357539788),
 (17, 0.006737816732479359),
 (18, 0.023421093956944444),
 (19, 0.015732928895791556),
 (20, 0.012468240369946643),
 (21, 0.005789258261892912),
 (22, 0.00507761123699525),
 (23, 0.009722824707280977),
 (24, 0.02045759485206009),
 (25, 0.010046067299207014),
 (26, 0.007186225878089409),
 (27, 0.01726744132904142),
 (28, 0.018957612853715593),
 (29, 0.039062213103843946),
 (30, 0.03905457675331173),
 (31, 0.0596552556152424),
 (32, 0.07029226595427983),
 (33, 0.013374701390131945),
 (34, 0.0),
 (35, 0.01989318873384059),
 (36, 0.017925350561715476),
 (37, 

What is happening here is closest match to our input movie that is "Avengers: " is "The Avengers" which has the index value of 16.  
we calculated similarity scores of each index  
similarity[index_of_the_movie] this means similarity[16]. this will return similarity score of 16th index and then it's comparing with all the indexs in the dataset.  

based on the similarity score we'll decide that which movies are more similar to user's input movie  

we can see that similarity score of 16th index is (16,0.999999999999999)  
user input was Avengers:  
closest movie to this is "The Avenger"  
index of it is  
so similarity score between user's input movie and closest movie has to be maximum which in this case is maximum  

here we can see one more thing is max score is 0.99999 not 1.000 why bcoz input was "Avengers" and there is no movies like "Avengers" closeset similar movies is "The Avenger" that is why similarity score is "0.9999999"



In [19]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

# x retun (index, similarity score) and we have to sort based on the similarity scores only so we'll take x[1]

sorted_similar_movies

[(3, 1.0000000000000002),
 (119, 0.5660667334084062),
 (65, 0.4739064762535151),
 (1196, 0.27736725217268715),
 (428, 0.2400101850101236),
 (95, 0.2063965588105755),
 (210, 0.20555642599666604),
 (1359, 0.20438773732168222),
 (96, 0.19782094170329564),
 (813, 0.18229768118495995),
 (1909, 0.16547268889307995),
 (1740, 0.16533541650783964),
 (1720, 0.16247882131300237),
 (1013, 0.16190990969111113),
 (437, 0.15201664906266354),
 (2793, 0.1457174405521284),
 (2371, 0.1442720301984286),
 (163, 0.1412944099718734),
 (72, 0.14041740504929656),
 (2790, 0.13528715820697174),
 (4664, 0.13385447999827005),
 (3373, 0.1334426746389728),
 (870, 0.13296630479411764),
 (1296, 0.13167566639882522),
 (1035, 0.1251229023704221),
 (299, 0.12402011193106896),
 (1033, 0.12382831205637275),
 (806, 0.12333587870550242),
 (303, 0.12177560661988926),
 (3172, 0.119943822692808),
 (10, 0.11991876777277595),
 (1234, 0.11826195096394622),
 (14, 0.11783709419157855),
 (873, 0.11489716695126793),
 (3431, 0.11401930

In [21]:
# printing top 30 similar movies based on the user's input

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if (i<31):     # in python index starts from 0
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . The Dark Knight Rises
2 . Batman Begins
3 . The Dark Knight
4 . The Prestige
5 . Batman Returns
6 . Interstellar
7 . Batman & Robin
8 . Batman
9 . Inception
10 . Superman
11 . Legend
12 . Kick-Ass 2
13 . Kick-Ass
14 . Child 44
15 . Get Smart
16 . The Killer Inside Me
17 . RockNRolla
18 . Watchmen
19 . Suicide Squad
20 . True Romance
21 . Bronson
22 . The Other Side of Heaven
23 . Superman II
24 . Superman III
25 . Jonah Hex
26 . Batman Forever
27 . Insomnia
28 . Captain Corelli's Mandolin
29 . Catwoman
30 . The Contender


YOu can see yourself how accurate the recommeded movies are

### MOVIES RECOMMENDATION SYSTEM

In [31]:
movie_name = input(" Enter the name of your favourite movie : ")

movies_list = df['title'].tolist() 

close_match = difflib.get_close_matches(movie_name, movies_list)

closest_match = close_match[0]

index_of_the_movie = df[df.title == closest_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie])) 

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if (i<31):     # in python index starts from 0
    print(i, '.',title_from_index)
    i+=1

 Enter the name of your favourite movie : G I Joe
Movies suggested for you : 

1 . G.I. Jane
2 . The Lord of the Rings: The Two Towers
3 . A Few Good Men
4 . The Lord of the Rings: The Return of the King
5 . Southland Tales
6 . Daylight
7 . For Colored Girls
8 . The Juror
9 . Crimson Tide
10 . Fantastic 4: Rise of the Silver Surfer
11 . Atlas Shrugged Part II
12 . Syriana
13 . The Elephant Man
14 . Red Planet
15 . Delgo
16 . The Perfect Wave
17 . Under Siege 2: Dark Territory
18 . Good
19 . March or Die
20 . Forsaken
21 . Critical Care
22 . Far from Men
23 . Appaloosa
24 . The Road
25 . Alatriste
26 . The Joneses
27 . White Squall
28 . Robin Hood
29 . Red Sky
30 . Enemy of the State
